In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip install gdown 

In [ ]:
!gdown --id 1-DhkgqWAd8jmrdFb4-AKjfGPQq2uGqwt

In [ ]:
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog

import cv2
import pycocotools.mask as mask_util
import numpy as np

In [ ]:
from sartorius_vis import show_predictions, show_zoomed

In [ ]:
register_coco_instances('sartorius_val',{},'../input/k-fold-crossvalidation-coco-dataset-generator/coco_cell_valid_fold4.json', 
                        '../input/sartorius-cell-instance-segmentation/')

val_ds = DatasetCatalog.get('sartorius_val')

In [ ]:
##Helper functions
def read_target_masks(item):
    enc_targs = list(map(lambda x:x['segmentation'], item['annotations']))
    #enc_targs = mask_util.frPyObjects(enc_targs, 520, 704)
    tars = torch.tensor(mask_util.decode(enc_targs)).cuda()
    return tars.permute(2,0,1).bool()

def get_preds_and_tars(item):
    im = cv2.imread(item['file_name'])
    pred = predictor(im) 
    pred_masks = pred['instances'].pred_masks
    tars = read_target_masks(item)
    return item['file_name'], pred_masks, tars


def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred_masks, targ):
   
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ['annotations']))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)


def score_all(model, ths, config):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config))
    cfg.INPUT.MASK_FORMAT='bitmask'
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
    cfg.MODEL.WEIGHTS = model
    cfg.TEST.DETECTIONS_PER_IMAGE = 1000
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    predictor = DefaultPredictor(cfg)

    scores = []
    
    for item in val_ds:
        im =  cv2.imread(item['file_name'])
        output = predictor(im)
        pred_classes = output['instances'].pred_classes.cpu().numpy().tolist()
        pred_class = max(set(pred_classes), key=pred_classes.count)
        take = output['instances'].scores >= ths[pred_class]
        pred_masks = output['instances'].pred_masks[take]
        pred_masks = pred_masks.cpu().numpy()
        
        sc = score(pred_masks, item)
        scores.append(sc)
        
    return np.mean(scores)

## Finally calculate score across all validation files:

In [ ]:
config1 = "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
config2 = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"

In [ ]:
score_all('./model_best.pth', [0.19, 0.39, 0.59], config2 )

In [ ]:
!pip install gdown 

In [ ]:
!gdown --id 1-DhkgqWAd8jmrdFb4-AKjfGPQq2uGqwt

In [ ]:
!gdown --id 1-C4Ri5TeH34pQrfOmEGDSPLSHzEYmawT

In [ ]:
score_all('../input/best-transfer-100k/model_best.pth',  [0.15, 0.35, 0.67])

In [ ]:
score_all('./model_best.pth',  [0.15, 0.35, 0.67])

In [ ]:
!cp  ../input/best-transfer-100k/model_best.pth ./model_x101.pth

In [ ]:
!mv ./model_best.pth ./model_cascade_pseudo.pth
!mv ./model_x101.pth ./model_x101_pseudo.pth

In [ ]:
config1 = "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
config2 = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"

In [ ]:
scores = []

In [ ]:
i=5/100; j=5/100 #; k=12/100

for num in range(16,25):
    k=num/100
    sc = score_all('./model_cascade_pseudo.pth',  [0.10+i, 0.30+j, 0.50+k])
    scores.append([sc, [0.10+i, 0.30+j, 0.50+k]] )

In [ ]:
scores.sort()
scores

In [ ]:
#0.73

In [ ]:
scores1= [] 

In [ ]:
i=5/100; j=5/100 #; k=12/100

for num in range(10,25):
    k=num/100
    sc = score_all('./model_cascade.pth',  [0.10+i, 0.30+j, 0.50+k], config1)
    scores1.append([sc, [0.10+i, 0.30+j, 0.50+k]] )

In [ ]:
scores1.sort()
scores1

In [ ]:
#0.62

In [ ]:
scores2= [] 

In [ ]:
scores2.sort()
scores2[-5:]

In [ ]:
scores3=[]

In [ ]:
i=5/100; j=5/100 #; k=12/100

for num in range(10,25):
    k=num/100
    sc = score_all('./model_x101.pth',  [0.10+i, 0.30+j, 0.50+k], config2)
    scores2.append([sc, [0.10+i, 0.30+j, 0.50+k]] )

In [ ]:
i=5/100; j=5/100 #; k=12/100

for num in range(20,30):
    k=num/100
    sc = score_all('./model_x101_pseudo.pth',  [0.10+i, 0.30+j, 0.50+k], config2)
    scores2.append([sc, [0.10+i, 0.30+j, 0.50+k]] )

In [ ]:
scores3.sort()
scores3[-5:]

In [ ]:
#0.67 or 

In [ ]:
!gdown --id 1-6kgnkZTilzbLO6BqoqhVZo5kXBzcpVe #mrcnn x101 f4 #lb = 0.307 std thss
!mv model_best.pth model_x101.pth
!gdown --id 1CVWavXTMyjdT8IoiAW4AuDoV1qiO8VaJ #cascade run0 f4 #lb = 0.31 std thss
!mv model_best.pth model_cascade.pth
'''!gdown --id 1-6_Etpic1uk49GVVF66w1jmxhShj1ALc #cascade run1 f4
!mv model_best.pth model_best_cascade2.pth'''